In [16]:
import requests
import re
import os
from bs4 import BeautifulSoup

In [ ]:
rss_feed_url = "https://lexfridman.com/feed/podcast/"

page = requests.get(rss_feed_url)
soup = BeautifulSoup(page.content, 'xml')

In [13]:
podcast_items = soup.find_all('item')
mp3_url = podcast_items[0].find('enclosure')['url']

In [14]:
mp3_file = requests.get(mp3_url)
with open('myfile.mp3', 'wb') as f:
    f.write(mp3_file.content)


In [22]:
os.mkdir('./downloads')

In [24]:
count = 0
for podcast in podcast_items:
    if count == 3:
        break
    title = podcast.find('title').text
    description = podcast.find('description').text
    mp3_url = podcast.find('enclosure')['url']

    if re.search('kanye', description, re.IGNORECASE ):
        mp3_file = requests.get(mp3_url)
        with open(f'./downloads/{title}.mp3', 'wb') as f:
            f.write(mp3_file.content)
        count += 1

In [25]:
## Transcribe Podcasts

In [30]:
api_key = os.environ['ASSEMBLY_AI_KEY']

In [ ]:
def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

In [31]:
headers = {
        "authorization": api_key, #"YOUR-API-TOKEN"
        "content-type": "application/json"
    }

file_names = os.listdir('./downloads')
output_ids = []
for file in file_names:
    file_path = f'./downloads/{file}'
    print(file)
    
    response = requests.post('https://api.assemblyai.com/v2/upload',
                            headers=headers,
                            data=read_file(file_path))
    


    endpoint = "https://api.assemblyai.com/v2/transcript"
    json = {
        "audio_url": response.json()['upload_url'],
        "audio_start_from": 300000,
        "audio_end_at": 400000,
}
    
    response = requests.post(endpoint, json=json, headers=headers)
    output_ids.append(response.json()['id'])



#336 – Ben Shapiro: Politics, Kanye, Trump, Biden, Hitler, Extremism, and War.mp3
{'upload_url': 'https://cdn.assemblyai.com/upload/61ba1df2-2b87-4b62-b543-0ac660fa742b'}
#365 – Sam Harris: Trump, Pandemic, Twitter, Elon, Bret, IDW, Kanye, AI & UFOs.mp3
{'upload_url': 'https://cdn.assemblyai.com/upload/3f5ba41a-29cb-4113-a14c-732c4e0f5e75'}
#338 – Chamath Palihapitiya: Money, Success, Startups, Energy, Poker & Happiness.mp3
{'upload_url': 'https://cdn.assemblyai.com/upload/84ad7de5-605a-4606-aa0c-783c1963035a'}


In [ ]:
import time

for transcript_id in output_ids:

    #you need to setup billing to access the usage reports, thats where you find the id for file in queue
    endpoint = "https://api.assemblyai.com/v2/transcript/:id"


    while True:
        response = requests.get(endpoint, headers=headers)
        status = response.json()['status']
        if status == 'completed' :
            print("Got Transcript")
            with open(f'./transcripts/{transcript_id}.txt', 'w') as f:
                f.write(response.json()['text'])
            break    
        print("Did Not Recieve Transcript: try again in 3 minutes")
        time.sleep(180)



